# Poincaré operators

In this tutorial, we show how Poincaré operators can be used to efficiently solve the mixed form of the Hodge Laplace problem. Letting $P \Lambda^k$ denote the Whitney forms of order $k$, this problem is posed as : find $(v, u) \in P \Lambda^{k - 1} \times P \Lambda^k$ such that
```math
\begin{align}
	(v, v')_\Omega - (u, dv')_\Omega &= \langle g, v' \rangle, &
	\forall v' &\in H \Lambda^{k - 1}, \\
	(dv, u')_\Omega + (du, du')_\Omega &= \langle f, u' \rangle, &
	\forall u' &\in H \Lambda^k.
\end{align}
```

In [this paper](https://arxiv.org/abs/2410.08830), we show how to construct decompositions $P \Lambda^k = \bar{P} \Lambda^k \oplus \mathring{P} \Lambda^k$ such that $d$ is invertible on $\bar{P} \Lambda^k$. In turn, the Hodge-Laplace problem can be solved in four sequential steps:
```math
\begin{align}
		(d\bar{v}, d\bar{v}')_\Omega
		&= \langle f, d\bar{v}' \rangle, &
		\forall \bar{v}' &\in \bar{P} \Lambda^{k - 1} \\
		(d\bar{w}_v, d\bar{w}')_\Omega
		&= \langle g, d\bar{w}' \rangle - (\bar{v}, d\bar{w}')_\Omega, &
		\forall \bar{w}' &\in \bar{P} \Lambda^{k - 2} \\
		(d\bar{u}, d\bar{u}')_\Omega
		&= \langle f, \bar{u}' \rangle - (d\bar{v}, \bar{u}')_\Omega, &
		\forall \bar{u}' &\in \bar{P} \Lambda^{k} \\
		(d\bar{v}_u, d\bar{v}')_\Omega
		&= (\bar{v} + d\bar{w}_v, \bar{v}')_\Omega
		- (\bar{u}, d\bar{v}')_\Omega
		- \langle g, \bar{v}' \rangle, & \forall \bar{v}' &\in \bar{P} \Lambda^{k - 1}.
\end{align}
```

In [31]:
import numpy as np
import scipy.sparse as sps
import time

import pygeon as pg
from pygeon.numerics.differentials import exterior_derivative as diff
from pygeon.numerics.innerproducts import mass_matrix

In [32]:
h = 0.1
dim = 3

# Grid generation
mdg = pg.unit_grid(dim, h)
pg.convert_from_pp(mdg)
mdg.compute_geometry()
sd = mdg.subdomains(dim=dim)[0]
print(sd)

# Create the Poincare object
poin = pg.Poincare(mdg)



Tetrahedral grid.
Number of cells 4613
Number of faces 9960
Number of nodes 1146



In [33]:
np.random.seed(0)

f_list = [None] * (dim + 1)
f_list[0] = np.random.rand(sd.num_nodes)
f_list[dim - 2] = np.random.rand(mdg.num_subdomain_ridges())
f_list[dim - 1] = np.random.rand(mdg.num_subdomain_faces())
f_list[dim] = np.random.rand(mdg.num_subdomain_cells())


# Assemble mass, differential, and stiffness matrices
M = [mass_matrix(mdg, dim - k, None) for k in range(dim + 1)] # (u, u')
D = [diff(mdg, dim - k) for k in range(dim)] # du
MD = [M[k + 1] @ D[k] for k in range(dim)] # (du, v')
S = [D[k].T @ MD[k] for k in range(dim)] # (du, du')
S.append(0)

# Subtract the mean from the 0-form right-hand side
f_list[0] -= np.sum(M[0] @ f_list[0]) / M[0].sum()

In [34]:
def timed_solve(A, b):
    t = time.time()
    sol = sps.linalg.spsolve(A.tocsc(), b)
    print("ndof: {}, Time: {:1.2f}s".format(len(b), time.time() - t))

    return sol


def solve_subproblem(poin, k, rhs):
    LS = pg.LinearSystem(S[k], rhs)
    LS.flag_ess_bc(~poin.bar_spaces[k], np.zeros_like(poin.bar_spaces[k]))

    return LS.solve(solver=timed_solve)

In [35]:
# Specify the order of u
k = 2

assert k >= 1  # There is no point in doing all of this for the 0-forms.

# Extract the right-hand sides from the randomly generated distributions
f = f_list[k]
g = f_list[k - 1]

# First, we perform a direct solve of the original problem
print("Full   |", end="")

# Assembly
saddle_point = sps.bmat([[M[k - 1], -MD[k - 1].T], [MD[k - 1], S[k]]])
LS = pg.LinearSystem(saddle_point, np.hstack((M[k - 1] @ g, M[k] @ f)))

# Solve the full system
full_sol = LS.solve(solver=timed_solve)

# Split the solution into v and u
v_full = full_sol[: g.size]
u_full = full_sol[g.size :]

print("----------------------------------")

# Second, we solve the problem in four steps

print("Step 1 |", end="") 
# Solve for bar{v}
v_bar = solve_subproblem(poin, k - 1, MD[k - 1].T @ f)

print("Step 2 |", end="") 
# Solve for bar{w}_v and set v = bar{v} + d bar{w}_v
if k >= 2:
    w_v_bar = solve_subproblem(poin, k - 2, MD[k - 2].T @ (g - v_bar))
    v = v_bar + D[k - 2] @ w_v_bar

else:  # k = 1.
    # There are no (k - 2)-forms, but we do need to subtract the mean of the solution.
    print("ndof: 0, Time: 0.00s")
    v = v_bar - np.sum(M[0] @ v_bar) / M[0].sum()

print("Step 3 |", end="") 
# Solve for bar{u}
u_bar = solve_subproblem(poin, k, M[k] @ f - MD[k - 1] @ v)

print("Step 4 |", end="") 
# Solve for bar{v}_u and set u = bar{u} + d bar{v}_u
v_u = solve_subproblem(poin, k - 1, M[k - 1] @ (v - g) - MD[k - 1].T @ u_bar)
u = u_bar + D[k - 1] @ v_u


assert np.allclose(v_full, v)
assert np.allclose(u_full, u)

Full   |ndof: 16452, Time: 1.96s
----------------------------------
Step 1 |ndof: 5347, Time: 0.10s
Step 2 |ndof: 1145, Time: 0.01s
Step 3 |ndof: 4613, Time: 0.00s
Step 4 |ndof: 5347, Time: 0.10s
